In [4]:
#シンチの全カウントのデータ,1分あたり。 efficiency測定のときの記録から
# sinci = [2351.747222, 1950.315605, 336.5333333, 336.2592593] ver2
sinci = [1152.864046, 1386.804167, 195, 336.7457023] #ver1

In [5]:
#シンチがとったコインシデンスの数/1s
import pandas as pd
sf = pd.read_csv('angel_data.csv', header=None)
df = pd.read_csv('curve_plot.csv', header=None)
num = 1

time = float(sf.at[num, 3])
minute = time/60
toukei = [int(sf.at[num, 8+i]) for i in range(5)]

In [6]:
#random coincidenceのシミュレーション

import numpy as np
rcoin_minute = [[] for _ in range(5)]

def random_coin(label, coin1, coin2):
    N = 100
    for j in range(N): #N回シミュレーションして、
            duration = 60  #秒、1分　<- ここを長くしすぎるととんでもない時間かかるので、1分にして後で直す
            n_a = int(sinci[coin1])       # 信号Aの数
            n_b = int(sinci[coin2])       # 信号Bの数
            width_a = 1e-6   # パルスAの幅（秒）
            width_b = 1e-6     # パルスBの幅（秒）

            # 信号の開始時刻をランダムにサンプリング
            start_times_a = np.random.uniform(0, duration, n_a)
            start_times_b = np.random.uniform(0, duration, n_b)

            # パルスの終了時刻を計算
            end_times_a = start_times_a + width_a
            end_times_b = start_times_b + width_b

            # コインシデンスのカウント
            coincidence_count = 0
            for k in range(n_a):
                a_start = start_times_a[k]
                a_end = end_times_a[k]
                
                # B信号との重なりをチェック
                overlaps = np.logical_and(start_times_b < a_end, end_times_b > a_start)
                coincidence_count += np.count_nonzero(overlaps)
            rcoin_minute[label].append(coincidence_count)

random_coin(0, 0, 2) #coin13
random_coin(1, 0, 3) #coin14
random_coin(2, 1, 2) #coin23
random_coin(3, 1, 3) #coin24
random_coin(4, 2, 3) #coin34   

In [7]:
#overlapの誤差の計算
rcoin_time = [np.average(rcoin_minute[i])*minute for i in range(5)]
print(rcoin_time)

error6_rcoin = [float(e/d) for e,d in zip(rcoin_time, toukei)]
print(error6_rcoin)

[np.float64(4.8), np.float64(0.0), np.float64(2.4), np.float64(2.4), np.float64(0.0)]
[0.00450281425891182, 0.0, 0.0005030391951372878, 0.00040140491721023583, 0.0]


In [9]:
#errorの保存
for i in range(5):
    df.at[num, i+52] = error6_rcoin[i]
df.to_csv('curve_plot.csv', index=False, header=False)